# AirBNB tracker

In [1]:
import numpy as np
import datetime
import pandas as pd
import pyperclip
import pickle
import re

In [2]:
tous_les_logements = pickle.load(open('tous_les_logements.p', 'rb'))
print(tous_les_logements.shape)
tous_les_logements.tail(2)

(1199, 12)


,date_enregistrement,texte,type,ville,description,prix,note,nb_avis,nb_lits,nb_chambres,est_professionnel,est_nouveau
1222,2024-03-02,Hébergement ⋅ Cachan\nMaison proche Paris avec...,Hébergement,Cachan,Maison proche Paris avec jardin,1105,NaN,NaN,NaN,NaN,False,True
1223,2024-03-02,Hébergement ⋅ L'Haÿ-les-Roses\nGrande Villa mo...,Hébergement,L'Haÿ-les-Roses,Grande Villa moderne et lumineuse avec piscine,2003,NaN,NaN,NaN,NaN,False,False


In [3]:
class Logement:
    __slots__ = [
        'date_enregistrement',
        'texte',
        'type',
        'ville',
        'description',
        'prix',
        'note',
        'nb_avis',
        'nb_lits',
        'nb_chambres',
        'est_professionnel',
        'est_nouveau'
    ]
    def __init__(self, ville: str, type: str, description: str,
                 nb_lits: int, nb_chambres: int,
                 est_professionnel: bool, est_nouveau: bool,
                 prix: int, note: float, nb_avis: int,
                 texte: str):
        self.type = type
        self.ville = ville
        self.description = description
        self.nb_lits = nb_lits
        self.nb_chambres = nb_chambres
        self.prix = prix
        self.est_nouveau = est_nouveau
        self.est_professionnel = est_professionnel
        self.note = note
        self.nb_avis = nb_avis

        self.date_enregistrement = datetime.date.today()
        self.texte = texte
    
    @classmethod
    def from_text(cls, texte):
        texte = texte.strip()
        attrs = set(cls.__slots__) - {'texte', 'date_enregistrement'}
        kwargs = dict(texte=texte)
        for attr in attrs:
            kwargs[attr] = eval(f'cls._get_{attr}')(texte)
        return cls(**kwargs)
    
    
    @staticmethod
    def _get_type(texte: str):
        pat = ' (⋅|·) '
        return re.split(pat, texte.split('\n')[0])[0]
        #return texte.split('\n')[0].split(pat)[0]
    
    @staticmethod
    def _get_ville(texte: str):
        pat = ' (⋅|·) '
        return re.split(pat, texte.split('\n')[0])[2]
        #return texte.split('\n')[0].split(' ⋅ ')[1]

    @staticmethod
    def _get_description(text: str):
        return text.split('\n')[1]
    
    @staticmethod
    def _get_nb_lits(texte: str):
        return Logement._get_nb_lits_nb_chambres(texte)[0]

    @staticmethod
    def _get_nb_chambres(texte: str):
        return Logement._get_nb_lits_nb_chambres(texte)[1]
    
    @staticmethod
    def _get_nb_lits_nb_chambres(texte: str) -> tuple:
        nb_lits, nb_chambres = None, None
        pat = r'(?P<nb_lits>\d) lits?'
        if (rgx := re.search(pat, texte.split('\n')[2])):
            nb_lits = int(rgx.group()[0])
        pat = r'(?P<nb_chambres>\d) chambres?'
        if (rgx := re.search(pat, texte.split('\n')[2])):
            nb_chambres = int(rgx.group()[0])
        return nb_lits, nb_chambres
    
    @staticmethod
    def _get_est_professionnel(texte: str):
        return texte.split('\n')[3] == 'Professionnel'
    
    @staticmethod
    def _get_prix(texte: str):
        for ligne in texte.split('\n'):
            if rgx := re.match('(?P<prix>[\d\u202f]+) € par nuit', ligne):
                return int(rgx.groups()[0].replace('\u202f', ''))
        raise ValueError('Pas de prix trouvé sur :\n' + texte)
    
    @staticmethod
    def _get_est_nouveau(texte: str):
        return texte.split('\n')[-1] == 'Nouveau'
    
    @staticmethod
    def _get_note(texte: str):
        return Logement._get_note_et_nb_avis(texte)[0]

    @staticmethod
    def _get_nb_avis(texte: str):
        return Logement._get_note_et_nb_avis(texte)[1]

    @staticmethod
    def _get_note_et_nb_avis(texte: str):
        note, nb_avis = None, None
        pat = r'(?P<note>\d\,\d\d?) \((?P<nb_avis>\d+)\)'
        if rgx := re.search(pat, texte):
            note = float(rgx.groupdict()['note'].replace(',', '.'))
            nb_avis = int(rgx.groupdict()['nb_avis'])
        return note, nb_avis
    
    def __repr__(self):
        rep = f'{self.__class__.__name__}('
        for attr in ('type', 'ville', 'prix'):
            rep += f'\n    {attr}={getattr(self, attr)},'
        rep = rep[:-1] + '\n)'
        return rep


def tronque_à_gauche(texte: str) -> str:
    """Tronque à gauche le texte avant 'Classement des résultats'"""
    stop = texte.find('Classement des résultats')
    return texte[stop + len('Classement des résultats'):].strip()

def tronque_à_droite(texte: str) -> str:
    """Tronque à droite le texte qui dépasse de la numérotation des pages"""
    # stop = texte.find('1\n2\n3\n4')
    stop = re.search(r'[\d…]\n[\d…]\n[\d…]', texte)
    if stop:
        return texte[:stop.start()].strip()
    stop = re.search(r'Carte Google \d+ séjours affichés.', texte)
    texte = texte[:stop.start()].strip()
    stop = re.search(r'Un voyage, deux logements', texte)
    if stop:
        texte = texte[:stop.start()].strip()
        
    pat = '\n(Appartement|Chambre|Hébergement|Tiny house|Appartement en résidence|Villa|Maison de ville) [⋅·] .+€\n'
    stop = re.search(pat, texte)
    if stop:
        texte = texte[:stop.start()].strip()
    return texte

In [105]:
page = pyperclip.paste()

page = tronque_à_gauche(page)
page = tronque_à_droite(page)

In [106]:
pat = "(Appartement|Chambre|Hébergement|Loft|Tiny house|Appartement en résidence|Villa|Maison de ville|Maison d'hôtes) [⋅·] "
splits = [
    match.start()
    for match in re.finditer(pat, page)
]
splits += [len(page)]

logements = []
for texte in [page[start:stop] for start, stop in zip(splits, splits[1:])]:
    logements.append(Logement.from_text(texte))

In [107]:
dico = [{attr: getattr(logement, attr) for attr in Logement.__slots__} for logement in logements]
page_df = pd.DataFrame(dico, index=range(len(dico)))

In [108]:
#for t in page_df.texte.unique(): print(t, '\n\n')
page_df.head(3)

,date_enregistrement,texte,type,ville,description,prix,note,nb_avis,nb_lits,nb_chambres,est_professionnel,est_nouveau
0,2024-03-10,Appartement ⋅ Villejuif\nBalcon des Horizons -...,Appartement,Villejuif,Balcon des Horizons - Appt 2 chambres avec Par...,402,NaN,NaN,None,None,False,True
1,2024-03-10,Appartement ⋅ Malakoff\n205. Mezzanine#4Pers#M...,Appartement,Malakoff,205. Mezzanine#4Pers#Malakoff,297,NaN,NaN,None,None,False,True
2,2024-03-10,Appartement ⋅ Montrouge\nCosy T2 aux portes de...,Appartement,Montrouge,Cosy T2 aux portes de Paris,325,NaN,NaN,None,None,False,True


In [109]:
assert page_df.shape[0] == 18
assert page_df.drop(['nb_lits', 'nb_chambres'], axis=1).notnull().sum().min() > 0

In [110]:
# On ajoute à tous les logements la page en cours de scrapping : 
tous_les_logements = pd.concat([tous_les_logements, page_df], axis=0, ignore_index=True)
print(tous_les_logements.shape)
tous_les_logements.tail()

(1487, 12)


,date_enregistrement,texte,type,ville,description,prix,note,nb_avis,nb_lits,nb_chambres,est_professionnel,est_nouveau
1482,2024-03-10,Appartement ⋅ Thiais\nChamant T2 proche aéropo...,Appartement,Thiais,Chamant T2 proche aéroport Orly et Paris,342,NaN,NaN,None,None,False,False
1483,2024-03-10,Appartement ⋅ Chevilly-Larue\nLogement cosy po...,Appartement,Chevilly-Larue,Logement cosy pour 6 personnes,462,NaN,NaN,None,None,False,True
1484,2024-03-10,Appartement ⋅ Bagneux\nCharmant appart proche ...,Appartement,Bagneux,Charmant appart proche Paris 10min Métro 4,407,4.45,22.0,None,None,False,False
1485,2024-03-10,"Appartement ⋅ Bagneux\nNEW : - 20%, 8/6/4 voya...",Appartement,Bagneux,"NEW : - 20%, 8/6/4 voyageurs",2168,NaN,NaN,None,None,False,False
1486,2024-03-10,Hébergement ⋅ Bourg-la-Reine\nBelle maison pro...,Hébergement,Bourg-la-Reine,Belle maison proche Paris,878,NaN,NaN,None,None,False,True


Quand on a fini d'itérer sur les pages, on dump :

In [111]:
pickle.dump(tous_les_logements.drop_duplicates().fillna(np.nan), open('tous_les_logements.p', 'wb'))